In [107]:
""" 
SOAR Offline Laptime Optimization for Reference Trajectory Generation
AA 203/273 Spring 2024
Andrew Wang, Bryan Yang
"""
import numpy as np
from pyomo.environ import *
from pyomo.dae import *
from agents import BicycleVehicle
from config import *

In [108]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:
# Set up track for optimization
veh_config = get_vehicle_config()
scene_config = get_scene_config()
veh_constraints = get_vehicle_opt_constraints(veh_config, scene_config)
track = scene_config["track"]

Track Closure Error [-1.33339491e-02 -4.81617196e-02  8.30401052e-08]


In [115]:
print(track.name)

OVAL w/Straight 100, Curve Radius 90


In [118]:
def compute_raceline(track, vehicle_config, vehicle_constraints):

    #SYSTEM STATES:  vx=x[0],  vy=x[1], wz=x[2] ,e_psi=x[3], t=x[4], e_y=x[5]
    #SYSTEM INPUTS:  ax[m/s^2]=u0, steering(rad)=u1
    #INDEPENDENT VARIABLE IS s (space)

    lf = vehicle_config["lf"]
    lr = vehicle_config["lr"]

    mass = vehicle_config["m"]
    Iz = vehicle_config["Iz"]

#     tire_model = vehicle_config.tire_model

#     mu = vehicle_config.wheel_friction
#     Br = vehicle_config.pacejka_b_rear
#     Bf = vehicle_config.pacejka_b_front
#     Cr = vehicle_config.pacejka_c_rear
#     Cf = vehicle_config.pacejka_c_front
#     Dr = vehicle_config.pacejka_d_rear
#     Df = vehicle_config.pacejka_d_front

#     xu_ub = vehicle_constraints['xu_ub']
#     xu_lb = vehicle_constraints['xu_lb']
#     du_ub = vehicle_constraints['du_ub']
#     du_lb = vehicle_constraints['du_lb']
    c_alpha_f = vehicle_config["c"]
    c_alpha_r = vehicle_config["c"]

    v_long_max = vehicle_constraints["ub_vx"]
    v_long_min = vehicle_constraints["lb_vx"]
    v_tran_max = vehicle_constraints["ub_vy"]
    v_tran_min = vehicle_constraints["lb_vy"]
    psidot_max = vehicle_constraints["ub_omega"]
    psidot_min = vehicle_constraints["lb_omega"]
    e_psi_max = vehicle_constraints["ub_epsi"]
    e_psi_min = vehicle_constraints["lb_epsi"]
    e_y_max = vehicle_constraints["ub_ey"]
    e_y_min = vehicle_constraints["lb_ey"]
    steer_max = vehicle_constraints["ub_delta"]
    steer_min = vehicle_constraints["lb_delta"]

    a_max = vehicle_config["max_accel"]
    a_min = -vehicle_config["max_accel"]
    steer_rate_max = vehicle_config["max_steer_rate"]
    steer_rate_min = -vehicle_config["max_steer_rate"]

#     a_rate_max = du_ub.u_a
#     a_rate_min = du_lb.u_a


    print('=================== Raceline computation ===================')
    print('Track name: %s' % track.name)
    print('     - length: %g m' % track.total_len)
    print('     - width: %g m' % (track.half_width*2))
    print('Vehicle configuration:')
    print('     - center of mass to front axle: %g m' % lf)
    print('     - center of mass to rear axle: %g m' % lr)
    print('     - mass: %g kg' % mass)
    print('     - yaw inertia: %g kg m^2' % Iz)
#     print('     - tire model: %s' % tire_model)
#     print('     - tire friction coefficient: %g' % mu)
#     print('     - Front tires:')
#     print('         - B: %g' % Bf)
#     print('         - C: %g' % Cf)
#     print('         - D: %g' % Df)
#     print('     - Rear tires:')
#     print('         - B: %g' % Br)
#     print('         - C: %g' % Cr)
#     print('         - D: %g' % Dr)
    print('Vehicle input constraints:')
    print('     - max acceleration: %g m/s^2' % a_max)
    print('     - min acceleration: %g m/s^2' % a_min)
    print('     - max steering angle: %g rad' % steer_max)
    print('     - min steering angle: %g rad' % steer_min)
    print('Vehicle state constraints:')
    print('     - max longitudinal velocity: %g m/s' % v_long_max)
    print('     - min longitudinal velocity: %g m/s' % v_long_min)
    print('     - max lateral velocity: %g m/s' % v_tran_max)
    print('     - min lateral velocity: %g m/s' % v_tran_min)
    print('     - max yaw rate: %g rad/s' % psidot_max)
    print('     - min yaw rate: %g rad/s' % psidot_min)
    print('============================================================')

    model = m = ConcreteModel()
    m.sf = Param(initialize=track.track_length)
    m.s = ContinuousSet(bounds=(0, m.sf))                             #s

    m.u0 = Var(m.s, bounds=(a_min, a_max), initialize=0)
#     m.u1 = Var(m.s, bounds=(steer_min, steer_max), initialize=0)
    m.u1 = Var(m.s, bounds=(steer_rate_min, steer_rate_max), initialize=0)
    m.alpha_f = Var(m.s, initialize=0)
    m.alpha_r = Var(m.s, initialize=0)
    m.Fyf = Var(m.s,bounds=(-mass*9.8, mass*9.8), initialize=0)
    m.Fyr = Var(m.s,bounds=(-mass*9.8, mass*9.8), initialize=0)
    m.x0 = Var(m.s, bounds=(v_long_min, v_long_max), initialize=0.01) #vx
    m.x1 = Var(m.s, bounds=(v_tran_min, v_tran_max), initialize=0)    #vy
    m.x2 = Var(m.s, bounds=(psidot_min, psidot_max), initialize=0)    #omega
    m.x3 = Var(m.s, bounds=(e_psi_min, e_psi_max))                    #epsi
    m.x4 = Var(m.s, bounds=(0, 20000), initialize=0)                  #t
    m.x5 = Var(m.s, bounds=(e_y_min, e_y_max), initialize=0)          #ey
    m.x6 = Var(m.s, bounds=(steer_min, steer_max), initialize=0)      #delta

    m.dx0ds = DerivativeVar(m.x0, wrt=m.s)
    m.dx1ds = DerivativeVar(m.x1, wrt=m.s)
    m.dx2ds = DerivativeVar(m.x2, wrt=m.s)
    m.dx3ds = DerivativeVar(m.x3, wrt=m.s)
    m.dx4ds = DerivativeVar(m.x4, wrt=m.s)
    m.dx5ds = DerivativeVar(m.x5, wrt=m.s)
    m.dx6ds = DerivativeVar(m.x6, wrt=m.s)
    m.du0ds = DerivativeVar(m.u0, wrt=m.s)
    m.du1ds = DerivativeVar(m.u1, wrt=m.s)

    # to avoid divide by 0
    eps=0.000001

    #Objective function
    m.obj = Objective(expr=m.x4[m.sf] + 0.01*sum(m.du0ds[i]**2 for i in m.s) + 0.01*sum(m.du1ds[i]**2 for i in m.s), sense=minimize)
    # ways to tune the cost:
        # penalize u1
        # penalize u2
        # penalize u1 and u2
        # vary the penalizations on u1 and u2
        # penalize the input rate
    # to do: write this optimization problem as a function that gets called
    # different arguments will choose a different objective tuning
    # function output should be the DF, then we can store them all

    #sideslip and lateral force
    def _alphafc(m, s):
#         return m.alpha_f[s] == m.u[s] - atan((m.x1[s] + lf * m.x2[s])/ (m.x0[s]))
        # alpha = delta - atan((vy+lf*w)/vx)
        return m.alpha_f[s] == m.x6[s] - atan((m.x1[s] + lf * m.x2[s])/ (m.x0[s]))
    m.c4 = Constraint(m.s, rule=_alphafc)
    def _alpharc(m, s):
        # alpha = - atan((vy-lr*w)/vx)
        return m.alpha_r[s] == -atan((m.x1[s] - lr * m.x2[s])/ (m.x0[s]))
    m.c3 = Constraint(m.s, rule=_alpharc)

#     if tire_model == 'linear':
#         def _Fyfc(m, s):
#             return m.Fyf[s] ==  Df * Cf * Bf * m.alpha_f[s]
#         m.c2 = Constraint(m.s, rule=_Fyfc)
#         def _Fyrc(m, s):
#             return m.Fyr[s] ==  Dr * Cr * Br * m.alpha_r[s]
#         m.c1 = Constraint(m.s, rule=_Fyrc)
#     elif tire_model == 'pacejka':
#         def _Fyfc(m, s):
#             return m.Fyf[s] ==  Df * sin(Cf * atan(Bf * m.alpha_f[s]))
#         m.c2 = Constraint(m.s, rule=_Fyfc)
#         def _Fyrc(m, s):
#             return m.Fyr[s] ==  Dr * sin(Cr * atan(Br * m.alpha_r[s]))
#         m.c1 = Constraint(m.s, rule=_Fyrc)
        
    def _Fyfc(m, s):
#         return m.Fyf[s] ==  Df * Cf * Bf * m.alpha_f[s]
        return m.Fyf[s] ==  c_alpha_f * m.alpha_f[s]
    m.c2 = Constraint(m.s, rule=_Fyfc)
    def _Fyrc(m, s):
#         return m.Fyr[s] ==  Dr * Cr * Br * m.alpha_r[s]
        return m.Fyr[s] ==  c_alpha_r * m.alpha_r[s]
    m.c1 = Constraint(m.s, rule=_Fyrc)

    #Differential model definition
    def _x0dot(m, s):
        cur = track.getCurvature(s)
        # print(cur)
        # dvx/ds = (a - (1/m*Fyfsin(delta) + omega*vy)*(1-kappa*ey) / (eps+vx*cos(epsi) - vy*sin(epsi))
        return m.dx0ds[s] == (m.u0[s] - 1 / mass *  m.Fyf[s] * sin(m.u1[s]) + m.x2[s]*m.x1[s])*((1 - cur * m.x5[s]) /(eps+m.x0[s] * cos(m.x3[s]) - m.x1[s] * sin(m.x3[s])))
    m.x0dot = Constraint(m.s, rule=_x0dot)

#     def _x1dot(m, s):
#         cur = track.get_curvature(s)
#         return m.dx1ds[s] == (1 / mass * (m.Fyf[s] * cos(m.u1[s]) + m.Fyr[s]) - m.x2[s] * m.x0[s])*((1 - cur * m.x5[s]) /(eps+m.x0[s] * cos(m.x3[s]) - m.x1[s] * sin(m.x3[s])))
#     m.x1dot = Constraint(m.s, rule=_x1dot)

#     def _x2dot(m, s):
#         cur = track.get_curvature(s)
#         return m.dx2ds[s] == (1 / Iz *(lf*m.Fyf[s] * cos(m.u1[s]) - lr * m.Fyr[s]) )*((1 - cur * m.x5[s]) /(eps+m.x0[s] * cos(m.x3[s]) - m.x1[s] * sin(m.x3[s])))
#     m.x2dot = Constraint(m.s, rule=_x2dot)

#     def _x3dot(m, s):
#         cur = track.get_curvature(s)
#         return m.dx3ds[s] == ( m.x2[s]*(1 - cur * m.x5[s])/(eps+m.x0[s] * cos(m.x3[s]) - m.x1[s] * sin(m.x3[s])) - cur)
#     m.x3dot = Constraint(m.s, rule=_x3dot)

#     def _x4dot(m, s):
#         cur = track.get_curvature(s)
#         return m.dx4ds[s] == ((1 - cur * m.x5[s]) /(eps+m.x0[s] * cos(m.x3[s]) - m.x1[s] * sin(m.x3[s])))
#     m.x4dot = Constraint(m.s, rule=_x4dot)

#     def _x5dot(m, s):
#         cur = track.get_curvature(s)
#         return m.dx5ds[s] == (m.x0[s] * sin(m.x3[s]) + m.x1[s] * cos(m.x3[s]))*((1 - cur * m.x5[s]) /(eps+m.x0[s] * cos(m.x3[s]) - m.x1[s] * sin(m.x3[s])))
#     m.x5dot = Constraint(m.s, rule=_x5dot)

#     # rate constraints on acceleration
#     # def _u0dotmax(m, s):
#     #     cur = track.get_curvature(s)
#     #     return m.du0ds[s] <= a_rate_max*((1 - cur * m.x5[s]) /(eps+m.x0[s] * cos(m.x3[s]) - m.x1[s] * sin(m.x3[s])))
#     # m.i0dotub = Constraint(m.s, rule=_u0dotmax)
#     # def _u0dotmin(m, s):
#     #     cur = track.get_curvature(s)
#     #     return m.du0ds[s] >= a_rate_min*((1 - cur * m.x5[s]) /(eps+m.x0[s] * cos(m.x3[s]) - m.x1[s] * sin(m.x3[s])))
#     # m.i0dotlb = Constraint(m.s, rule=_u0dotmin)

#     # rate constraints on steering
# #     def _u1dotmax(m, s):
# #         cur = track.get_curvature(s)
# #         return m.du1ds[s] <= steer_rate_max*((1 - cur * m.x5[s]) /(eps+m.x0[s] * cos(m.x3[s]) - m.x1[s] * sin(m.x3[s])))
# #     m.i1dotub = Constraint(m.s, rule=_u1dotmax)
# #     def _u1dotmin(m, s):
# #         cur = track.get_curvature(s)
# #         return m.du1ds[s] >= steer_rate_min*((1 - cur * m.x5[s]) /(eps+m.x0[s] * cos(m.x3[s]) - m.x1[s] * sin(m.x3[s])))
# #     m.i1dotlb = Constraint(m.s, rule=_u1dotmin)

#     # inital and terminal conditions
#     def _init(m):
#         # yield m.x5[0] == m.x5[track.track_length]
#         yield m.x0[0] == m.x0[track.track_length]
#         yield m.x1[0] == m.x1[track.track_length]
#         yield m.x2[0] == m.x2[track.track_length]
#         yield m.x3[0] == m.x3[track.track_length]
#         yield m.x4[0] == 0
#         yield m.x5[0] == m.x5[track.track_length]
#     m.init_conditions = ConstraintList(rule=_init)

#     # Discretize model using radau or finite difference collocation
#     TransformationFactory('dae.collocation').apply_to(m, nfe=200, ncp=10, scheme='LAGRANGE-LEGENDRE') #STANDARD METHOD

#     # Solve algebraic model
#     solver = SolverFactory('ipopt')
#     # Solver options
#     solver.options['max_iter'] = 8000
#     results = solver.solve(m,tee=True)

#     # Evaluate solution at discretization points
#     s_vec = list(sorted(m.s.data()))

#     raceline = {'t': np.zeros(len(s_vec)),
#                 'x' : np.zeros(len(s_vec)),
#                 'y' : np.zeros(len(s_vec)),
#                 'psi' : np.zeros(len(s_vec)),
#                 's' : np.zeros(len(s_vec)),
#                 'e_y' : np.zeros(len(s_vec)),
#                 'e_psi' : np.zeros(len(s_vec)),
#                 'v_long' : np.zeros(len(s_vec)),
#                 'v_tran' : np.zeros(len(s_vec)),
#                 'psidot' : np.zeros(len(s_vec)),
#                 'u_a': np.zeros(len(s_vec)),
#                 'u_s': np.zeros(len(s_vec))}

#     for j in range(len(s_vec)):
#         s = s_vec[j]

#         local_pos = (s, value(m.x5[s]), 0)
#         (x, y, psi) = track.local_to_global(local_pos)

#         raceline['t'][j] = value(m.x4[s])
#         raceline['v_long'][j] = value(m.x0[s])
#         raceline['v_tran'][j] = value(m.x1[s])
#         raceline['psidot'][j] = value(m.x2[s])
#         raceline['s'][j] = s
#         raceline['e_y'][j] = value(m.x5[s])
#         raceline['e_psi'][j] = value(m.x3[s])
#         raceline['x'][j] = x
#         raceline['y'][j] = y
#         raceline['psi'][j] = psi
#         raceline['u_a'][j] = value(m.u0[s])
#         raceline['u_s'][j] = value(m.u1[s])

#     return raceline

def plot_racelines(track_name, raceline):
    import matplotlib.pyplot as plt
    import os

    track = get_track(track_name)

    fig_xy = plt.figure(figsize=(20, 20))
    ax = fig_xy.gca()
    track.plot_map(ax)

    fig_ts = plt.figure(figsize=(50, 30))
    ax_vx = fig_ts.add_subplot(6, 1, 1)
    ax_vx.set_ylabel('vel long')
    ax_vy = fig_ts.add_subplot(6, 1, 2)
    ax_vy.set_ylabel('vel lat')
    ax_pd = fig_ts.add_subplot(6, 1, 3)
    ax_pd.set_ylabel('yaw rate')
    ax_ey = fig_ts.add_subplot(6, 1, 4)
    ax_ey.set_ylabel('e_y')
    ax_a = fig_ts.add_subplot(6, 1, 5)
    ax_a.set_ylabel('accel cmd')
    ax_s = fig_ts.add_subplot(6, 1, 6)
    ax_s.set_ylabel('steer cmd')
    ax_s.set_xlabel('s')

    vs = np.cos(raceline['v_long']) - np.sin(raceline['v_tran'])
    speed = np.sqrt(np.power(raceline['v_long'],2)+np.power(raceline['v_tran'],2))
    sc = ax.scatter(raceline['x'], raceline['y'], c=speed)
    ax.plot(raceline['x'], raceline['y'], 'b')

    ax_vx.plot(raceline['s'], raceline['v_long'], 'b')
    ax_vy.plot(raceline['s'], raceline['v_tran'], 'b')
    ax_pd.plot(raceline['s'], raceline['psidot'], 'b')
    ax_ey.plot(raceline['s'], raceline['e_y'], 'b')
    ax_a.plot(raceline['s'], raceline['u_a'], 'b')
    ax_s.plot(raceline['s'], raceline['u_s'], 'b')

    laptime = raceline['t'][-1]
    print(f'Lap time: {laptime}')

    ax.set_aspect('equal')
    plt.colorbar(mappable=sc, ax=ax)
    ax.set_title(f'time: {laptime:.2f} s')

    plt.draw()

In [119]:
compute_raceline(track, veh_config, veh_constraints)

=================== Raceline computation ===================
Track name: OVAL w/Straight 100, Curve Radius 90
     - length: 765.487 m
     - width: 20 m
Vehicle configuration:
     - center of mass to front axle: 0.125 m
     - center of mass to rear axle: 0.125 m
     - mass: 2 kg
     - yaw inertia: 0.03 kg m^2
Vehicle input constraints:
     - max acceleration: 5 m/s^2
     - min acceleration: -5 m/s^2
     - max steering angle: 0.5 rad
     - min steering angle: -0.5 rad
Vehicle state constraints:
     - max longitudinal velocity: 60 m/s
     - min longitudinal velocity: 0 m/s
     - max lateral velocity: 10 m/s
     - min lateral velocity: -10 m/s
     - max yaw rate: 1 rad/s
     - min yaw rate: -1 rad/s


In [64]:
# Initialize upper and lower bounds on states
# x = [s, ey, epsi, vx, vy, omega, delta]
# u = [a, delta_dot]
track_halfwidth = scene_config["track_config"]["track_half_width"]
epsi_bounds = np.pi/2
vx_bounds = 30
vy_bounds = 10
omega_bounds = 1

lbx = [0., -track_halfwidth, -epsi_bounds, -vx_bounds, -vy_bounds, -omega_bounds, -veh_config["max_steer"]]
ubx = [1.2*track.total_len, track_halfwidth, epsi_bounds, vx_bounds, vy_bounds, omega_bounds, veh_config["max_steer"]]
lbu = [-veh_config["max_accel"], -veh_config["max_steer_rate"]]
ubu = [veh_config["max_accel"], veh_config["max_steer_rate"]]
lbT = [0]
ubT = [100]

In [85]:
# Define Optimization Problem in CasADi
N = 10
STATE_DIM = 7
CONTROL_DIM = 2
x0 = [0]*STATE_DIM

x = SX.sym('x', N+1, STATE_DIM)
u = SX.sym('u', N, CONTROL_DIM)
T = SX.sym('T', 1)

dt = T/N

# Declare arrays g = constraints with lower and upper bounds
g = []
lbg = []
ubg = []

agent = BicycleVehicle(veh_config, scene_config, None, None)

for i in range(N):
    g += [x[i+1,:] - agent.casadi_dynamics(x[i,:], u[i,0], u[i,1], dt)]
    lbg += [0]*7
    ubg += [0]*7
    
w = [x[0,:], T]





w = horzcat(*w)


lbw = lbx + lbT
ubw = lbx + ubT

print(lbw)


nlp = {'x':w, 'f':T+1, 'p':[], 'g':[]}
solver = nlpsol('nlp_solver', 'ipopt', nlp)
sol = solver(lbx=lbw, ubx=ubw)
print(sol['x'])

# for i in range(N):
#     print(x)
    

here first


Exception: Implicit conversion of symbolic CasADi type to numeric matrix not supported.
This may occur when you pass a CasADi object to a numpy function.
Use an equivalent CasADi function instead of that numpy function.